In [1]:
# Requires brainvisa

In [2]:
import anatomist.api as anatomist
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

In [3]:
from soma import aims
import json
import sys
import pandas as pd
import scipy
import scipy.stats
import numpy as np
import glob

In [4]:
a = anatomist.Anatomist()

global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module gltf_io
loading module palettecontrols
loading module paletteViewer
loading module meshsplit
loading module profilewindow
loading module ana_image_math
loading module anacontrolmenu
loading module foldsplit
loading module modelGraphs
loading module bsa_proba
loading module histogram
loading module gradientpalette
loading module infowindow
loading module simple_controls
loading module volumepalettes
loading module statsplotwindow
loading module save_resampled
loading module valuesplotwindow
loading module selection
all python modules loaded
Anatomist started.


In [27]:
Rspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg"

Lspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_left/meshes/Lspam_model_meshes_1.arg"


json_regions = "/neurospin/dico/data/deep_folding/current/sulci_regions_gridsearch.json"

#path_summary = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0/UKB-RAP"
#path_file = "all_regions.csv"
path_summary = '/neurospin/dico/data/deep_folding/current/models/Champollion_V1_after_ablation/'
path_file = 'p_values_asd.csv'


In [28]:
with open(json_regions) as f:
    regions = json.load(f)

In [29]:
file_to_display = f"{path_summary}/{path_file}"

In [30]:
next(iter(regions['brain']))

'S.C.-sylv._left'

In [31]:
df = pd.read_csv(file_to_display)[["region", "p"]]
df["p"] = -np.log10(df["p"])
df.head()

,region,p
0,FCLp-subsc-FCLa-INSULA_right,6.866671
1,STi-SOTlat_left,4.347952
2,STi-STs-STpol_right,3.919829
3,OCCIPITAL_right,3.911627
4,FColl-SRh_left,3.532080


In [32]:

res = df.groupby(['region']).mean()
res["side"] = res.index.str.split('_').str[-1]
res = res.reset_index()

In [33]:
res = res[~res.region.str.contains("fronto-parietal")]
res = res[~res.region.str.contains("STs-SGSM")]
res = res[~res.region.str.contains("FCLp-SGSM")]
res = res[~res.region.str.contains("CINGULATE")]

In [34]:
"CINGULATE_left".replace("CINGULATE", "CINGULATE.")

'CINGULATE._left'

In [35]:
def get_sulci(region):
    region = region.replace("CINGULATE", "CINGULATE.")
    region = region.replace("ORBITAL", "S.Or.")
    list_sulci = list(regions['brain'][f"{region}"].keys())
    list_sulci = [x.replace("paracingular.", "S.F.int.") for x in list_sulci]
    return list_sulci

In [36]:
get_sulci("CINGULATE_left")

['S.F.int._left', 'F.C.M.ant._left']

In [37]:
# res["region"].tolist()

In [38]:
dic = {
    "FCLp-subsc-FCLa-INSULA": "F.C.L.p.-subsc.-F.C.L.a.-INSULA.",
    "FCMpost-SpC": "F.C.M.post.-S.p.C.",
    "FColl-SRh": "F.Coll.-S.Rh.",
    "FIP": "F.I.P.",
    "FPO-SCu-ScCal": "F.P.O.-S.Cu.-Sc.Cal.",
    "Lobule_parietal_sup": "Lobule_parietal_sup.",
    "SC-SPeC": "S.C.-S.Pe.C.",
    "SC-SPoC": "S.C.-S.Po.C.",
    "SC-sylv": "S.C.-sylv.",
    "SFinf-BROCA-SPeCinf": "S.F.inf.-BROCA-S.Pe.C.inf.",
    "SFint-FCMant": "S.F.int.-F.C.M.ant.",
    "SFint-SR": "S.F.int.-S.R.",
    "SFinter-SFsup": "S.F.inter.-S.F.sup.",
    "SFmarginal-SFinfant":"S.F.marginal-S.F.inf.ant.",
    "SFmedian-SFpoltr-SFsup": "S.F.median-S.F.pol.tr.-S.F.sup.",
    "SOr-SOlf": "S.Or.-S.Olf.",
    "SOr": "S.Or.",
    "SPeC": "S.Pe.C.",
    "SPoC": "S.Po.C.",
    "STi-SOTlat": "S.T.i.-S.O.T.lat.",
    "STi-STs-STpol": "S.T.i.-S.T.s.-S.T.pol.",
    "STsbr": "S.T.s.br.",
    "STs": "S.T.s.",
    "ScCal-SLi": "Sc.Cal.-S.Li.",
    "SsP-SPaint": "S.s.P.-S.Pa.int.",
}

In [39]:
for key in dic.keys():
    res["region"] = res["region"].str.replace(key, dic[key])
res.head()

,region,p,side
2,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,2.741967,left
3,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,6.866671,right
4,F.C.M.post.-S.p.C._left,0.379032,left
5,F.C.M.post.-S.p.C._right,0.502011,right
6,F.Coll.-S.Rh._left,3.532080,left


In [40]:
res['sulcus'] = res.apply(lambda x: get_sulci(x.region), axis=1)
res.head()

,region,p,side,sulcus
2,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,2.741967,left,"[F.C.L.p._left, F.C.L.r.sc.ant._left, F.C.L.r...."
3,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,6.866671,right,"[F.C.L.p._right, F.C.L.r.sc.ant._right, F.C.L...."
4,F.C.M.post.-S.p.C._left,0.379032,left,"[F.C.M.post._left, S.C.LPC._left, S.p.C._left]"
5,F.C.M.post.-S.p.C._right,0.502011,right,"[F.C.M.post._right, S.C.LPC._right, S.p.C._right]"
6,F.Coll.-S.Rh._left,3.532080,left,"[F.Coll._left, S.Rh._left]"


In [41]:
res = res.sort_values(by="p", ascending=False)
res = res.explode("sulcus")
res[res.region.str.contains("S.T.s.")]

,region,p,side,sulcus
47,S.T.i.-S.T.s.-S.T.pol._right,3.919829,right,S.T.i.ant._right
47,S.T.i.-S.T.s.-S.T.pol._right,3.919829,right,S.T.i.post._right
47,S.T.i.-S.T.s.-S.T.pol._right,3.919829,right,S.T.s._right
47,S.T.i.-S.T.s.-S.T.pol._right,3.919829,right,S.T.pol._right
46,S.T.i.-S.T.s.-S.T.pol._left,2.636611,left,S.T.i.ant._left
46,S.T.i.-S.T.s.-S.T.pol._left,2.636611,left,S.T.i.post._left
46,S.T.i.-S.T.s.-S.T.pol._left,2.636611,left,S.T.s._left
46,S.T.i.-S.T.s.-S.T.pol._left,2.636611,left,S.T.pol._left
49,S.T.s._right,2.512688,right,S.T.s._right
51,S.T.s.br._right,1.375297,right,F.I.P.r.int.1_right


In [42]:
for _, row in res.iterrows():
    print(row.sulcus)

F.C.L.p._right
F.C.L.r.sc.ant._right
F.C.L.r.sc.post._right
INSULA_right
F.C.L.a._right
S.T.i.ant._left
S.T.i.post._left
S.O.T.lat.ant._left
S.O.T.lat.int._left
S.O.T.lat.med._left
S.O.T.lat.post._left
S.T.i.ant._right
S.T.i.post._right
S.T.s._right
S.T.pol._right
OCCIPITAL_right
F.Coll._left
S.Rh._left
F.Coll._right
S.Rh._right
F.I.P._right
S.Po.C.sup._right
S.Pa.sup._right
S.Or._right
S.Olf._right
F.C.L.p._left
F.C.L.r.sc.ant._left
F.C.L.r.sc.post._left
INSULA_left
F.C.L.a._left
S.T.i.ant._right
S.T.i.post._right
S.O.T.lat.ant._right
S.O.T.lat.int._right
S.O.T.lat.med._right
S.O.T.lat.post._right
F.I.P._right
S.T.i.ant._left
S.T.i.post._left
S.T.s._left
S.T.pol._left
S.Or._left
S.Olf._left
F.I.P._left
S.Po.C.sup._left
S.Pa.sup._left
S.T.s._right
S.F.inf._right
S.F.inf.ant._right
F.C.L.r.ant._right
F.C.L.r.asc._right
F.C.L.r.diag._right
S.Pe.C.inf._right
S.Or._right
F.I.P._left
F.P.O._right
S.Pa.t._right
S.Cu._right
F.Cal.ant.-Sc.Cal._right
S.O.p._right
F.Cal.ant.-Sc.Cal._left
S.O.p._

In [43]:
res[res.sulcus=="S.F.orbitaire._right"]

,region,p,side,sulcus
33,S.F.marginal-S.F.inf.ant._right,0.886989,right,S.F.orbitaire._right


In [44]:
res

,region,p,side,sulcus
3,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,6.866671,right,F.C.L.p._right
3,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,6.866671,right,F.C.L.r.sc.ant._right
3,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,6.866671,right,F.C.L.r.sc.post._right
3,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,6.866671,right,INSULA_right
3,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,6.866671,right,F.C.L.a._right
...,...,...,...,...
26,S.F.int.-F.C.M.ant._left,0.085370,left,F.C.M.ant._left
26,S.F.int.-F.C.M.ant._left,0.085370,left,S.F.int._left
28,S.F.int.-S.R._left,0.046414,left,S.F.int._left
28,S.F.int.-S.R._left,0.046414,left,S.R.inf._left


In [62]:
def set_color_property(res, side):
    global dic

    if side == "L":
        spam_model_file = Lspam_model
    else:
        spam_model_file = Rspam_model
        
    dic[f"aims{side}"] = aims.read(spam_model_file)

    for vertex in dic[f"aims{side}"].vertices():
        vertex['p_value'] = 0.

    for _, row in res.iterrows():
        for vertex in dic[f"aims{side}"].vertices():
            vname = vertex.get('name')
            if vname == row.sulcus:
                    vertex['p_value'] = max(vertex['p_value'],
                                                row.p)
                    if row.p < -np.log10(0.05/56):
                        vertex['p_value'] = 0.
                    elif vertex['p_value'] != 0.: 
                        vertex['p_value'] = min(vertex['p_value'],
                                                row.p)
                    else:
                        vertex['p_value'] = row.p
    
    dic[f"ana{side}"] = a.toAObject(dic[f"aims{side}"])

    dic[f"ana{side}"].setColorMode(dic[f"ana{side}"].PropertyMap)
    dic[f"ana{side}"].setColorProperty('p_value')
    dic[f"ana{side}"].notifyObservers()
    
                
def visualize_whole_hemisphere(view_quaternion, side, i):
    global block
    global dic
    try:
        block
    except NameError:
        block = a.createWindowsBlock(4)

    dic[f"win{i}"] = a.createWindow('3D',
                                    block=block,
                                    no_decoration=True,
                                    options={'hidden': 1})
    dic[f"win{i}"].addObjects(dic[f"ana{side}"])
    dic[f"ana{side}"].setPalette("green_yellow_red",
                              minVal=-np.log10(0.05/56), maxVal=10,
                              absoluteMode=True)
    
    dic[f"win{i}"].camera(view_quaternion=view_quaternion)

middle_view = [0.5, -0.5, -0.5, 0.5]
side_view = [0.5, 0.5, 0.5, 0.5]
bottom_view = [0, -1, 0, 0]
top_view = [0, 0, 0, -1]
 
def visualize_whole(res, side, start):
    set_color_property(res, side)
    visualize_whole_hemisphere(middle_view if side == "L" else side_view, side, start+0)
    visualize_whole_hemisphere(top_view, side, start+1)
    visualize_whole_hemisphere(bottom_view, side, start+2)
    visualize_whole_hemisphere(side_view if side == "L" else middle_view, side, start+3)


In [63]:
dic = {}

visualize_whole(res, "L", 0)
visualize_whole(res, "R", 4)

Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60
Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60


no position could be read at 316, 409
no position could be read at 48, 352
no position could be read at 41, 108


: 